In [3]:
import yaml

# Import directory setting and volumes to mount from machine.yaml
with open(r'machine.yaml') as file:
    machinecfg = yaml.load(file, Loader=yaml.FullLoader)

# Transform volumes dict into "host=bind" strings 
volume_list = []
for key, value in machinecfg['volumes'].items():
    volume_list += [f'{value}:/{key}']

# Import parameters needed to render templates
with open(r'diagnostic.yaml') as file:
    diagcfg = yaml.load(file, Loader=yaml.FullLoader)



In [4]:
from jinja2 import Template, FileSystemLoader, Environment
import os

templateLoader = FileSystemLoader(searchpath="./templates")
templateEnv = Environment(loader=templateLoader)

# Now use jinja2 to render al top-level keys in recipe file
# as templates into actual recipes, config files etc.

for k in diagcfg:
    dest_file = os.path.join(machinecfg['volumes'][diagcfg[k]['target_volume']], diagcfg[k]['filename'])
    template_file = diagcfg[k]['template']
    template = templateEnv.get_template(template_file)
    template.stream(diagcfg[k]['parameters']).dump(dest_file)
    

In [5]:
# Run actual diagnostic docker

import docker
client = docker.from_env()
output = client.containers.run(image=machinecfg['image'],
                      command=machinecfg['command'], 
                      volumes=volume_list, 
                      environment=machinecfg['environment'])


In [6]:
# Parse the output to find results and plots

outputl = output.split(b'\n')
workdir = [i for i in outputl if b"miles_output" in i][1].decode("utf-8").rsplit('/', 1)[0]
plotdir = [i for i in outputl if b"miles_output" in i][-1].decode("utf-8").rsplit('/', 1)[0]
dict_out={'workdir': workdir, 'plotdir': plotdir}

# Write simple yaml containing information on output directories
with open(r'output.yaml', 'w') as file:
    documents = yaml.dump(dict_out, file)